## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,57.47,62,26,2.35,scattered clouds
1,1,Severo-Kurilsk,RU,50.6789,156.1250,23.16,68,26,11.92,scattered clouds
2,2,Bathsheba,BB,13.2167,-59.5167,79.41,65,75,19.57,broken clouds
3,3,Souillac,MU,-20.5167,57.5167,81.36,89,75,11.50,broken clouds
4,4,Tombouctou,ML,20.0000,-3.0000,61.83,20,16,11.63,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [106]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Bathsheba,BB,13.2167,-59.5167,79.41,65,75,19.57,broken clouds
3,3,Souillac,MU,-20.5167,57.5167,81.36,89,75,11.50,broken clouds
5,5,Faanui,PF,-16.4833,-151.7500,80.76,74,100,14.63,light rain
6,6,Hithadhoo,MV,-0.6000,73.0833,82.22,72,83,11.03,broken clouds
11,11,West Bay,KY,19.3667,-81.4167,83.07,71,75,2.30,broken clouds
13,13,Mana,GF,5.6592,-53.7767,76.64,89,31,9.26,scattered clouds
20,20,Kapaa,US,22.0752,-159.3190,79.12,64,75,14.97,broken clouds
23,23,Guerrero Negro,MX,27.9769,-114.0611,82.27,18,15,6.87,few clouds
24,24,Alugan,PH,12.2188,125.4808,78.06,81,100,9.48,overcast clouds
25,25,Tarakan,ID,3.3000,117.6333,76.50,94,75,0.00,broken clouds


In [107]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                193
City                   193
Country                192
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [108]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [109]:
preferred_cities_df.dropna(subset = ["Country"], inplace=True)

/Users/anabisker/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [110]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [111]:
preferred_cities_df.count()

City_ID                192
City                   192
Country                192
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [112]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167,
3,Souillac,MU,81.36,broken clouds,-20.5167,57.5167,
5,Faanui,PF,80.76,light rain,-16.4833,-151.7500,
6,Hithadhoo,MV,82.22,broken clouds,-0.6000,73.0833,
11,West Bay,KY,83.07,broken clouds,19.3667,-81.4167,
13,Mana,GF,76.64,scattered clouds,5.6592,-53.7767,
20,Kapaa,US,79.12,broken clouds,22.0752,-159.3190,
23,Guerrero Negro,MX,82.27,few clouds,27.9769,-114.0611,
24,Alugan,PH,78.06,overcast clouds,12.2188,125.4808,
25,Tarakan,ID,76.50,broken clouds,3.3000,117.6333,


In [113]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotel_response = requests.get(base_url, params=params)
    response_status = hotel_response.status_code
    
#     if response_status == HTTP_STATUS_OK:
#         hotels = hotel_response.json()
#         hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
#     elif response_status == HTTP_STATUS_NOT_FOUND:
#         print(f"{hotels}not found... skipping.")
#     else:
#         raise ValueError(f"HTTP status: {response_status} for {index}")
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotels = hotel_response.json()
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"In {row['City']} index {index} hotel not found... skipping.")

In Ligayan index 155 hotel not found... skipping.
In Yuli index 196 hotel not found... skipping.
In Luba index 199 hotel not found... skipping.
In Nioro index 226 hotel not found... skipping.
In Jutai index 354 hotel not found... skipping.
In Gayeri index 629 hotel not found... skipping.
In Thinadhoo index 643 hotel not found... skipping.


In [114]:
# 7. Drop the rows where there is no Hotel Name.
skipped_row_index = hotel_df["Hotel Name"] != ""
hotel_df = hotel_df.loc[skipped_row_index]
hotel_df.count()

City                   185
Country                185
Max Temp               185
Current Description    185
Lat                    185
Lng                    185
Hotel Name             185
dtype: int64

In [119]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bathsheba,BB,79.41,broken clouds,13.2167,-59.5167,Atlantis Hotel
3,Souillac,MU,81.36,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
5,Faanui,PF,80.76,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
6,Hithadhoo,MV,82.22,broken clouds,-0.6000,73.0833,Scoop Guest House
11,West Bay,KY,83.07,broken clouds,19.3667,-81.4167,Calypso Cove
13,Mana,GF,76.64,scattered clouds,5.6592,-53.7767,LE SAMANA
20,Kapaa,US,79.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
23,Guerrero Negro,MX,82.27,few clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
24,Alugan,PH,78.06,overcast clouds,12.2188,125.4808,JM Lodge
25,Tarakan,ID,76.50,broken clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan


In [115]:
# 8a. Create the output File (CSV)
output_data_path = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_path, index_label="City_ID")

In [116]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [117]:
# 11a. Add a marker layer for each city to the map. 
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))